<a href="https://colab.research.google.com/github/jems2019/AlternusVera/blob/socialCredibility/Twitter_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Setup and Library Imports


In [0]:
import pandas as pd
import numpy as np
import csv
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from string import punctuation
import seaborn as sns
import pandas as pd
import numpy as np
import nltk
import re
import nltk
import nltk.sentiment
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('wordnet')
import matplotlib.pyplot as plt
from scipy import sparse

#helper libraries to take metrics and split data
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.utils.multiclass import unique_labels
from scipy.spatial.distance import cdist


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Read the test, training and valid data from files
# Header = 0 indicates that the first line of the file contains column names,
# As there is no Header, create a column names for each column in the dataset
# delimiter = \t indicates that the fields are seperated by tabs, and 


test_filename = 'https://raw.githubusercontent.com/jems2019/AlternusVera/socialCredibility/input_data/dataset/test.tsv'
train_filename = 'https://raw.githubusercontent.com/jems2019/AlternusVera/socialCredibility/input_data/dataset/train.tsv'
valid_filename = 'https://raw.githubusercontent.com/jems2019/AlternusVera/socialCredibility/input_data/dataset/valid.tsv'

colnames = ['jsonid', 'label', 'headline_text', 'subject', 'speaker', 'speakerjobtitle', 'stateinfo','partyaffiliation', 'barelytruecounts', 'falsecounts','halftruecounts','mostlytrueocunts','pantsonfirecounts','context']

train_news = pd.read_csv(train_filename, sep='\t', names = colnames, error_bad_lines=False)
test_news = pd.read_csv(test_filename, sep='\t', names = colnames, error_bad_lines=False)
valid_news = pd.read_csv(valid_filename, sep='\t', names = colnames, error_bad_lines=False)

In [4]:
# Display check the dimensions and the first 2 rows of the file.

print('train dim:',train_news.shape, 'test dim:', test_news.shape)
train_news.iloc[0:2]

train dim: (10240, 14) test dim: (1267, 14)


,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.


# Setting up Twitter

In [0]:
# source: https://stackabuse.com/accessing-the-twitter-api-with-python/
import json

# Enter your keys/secrets as strings in the following fields
credentials = {}  

## CREDENTIALS 1 ##
# credentials['CONSUMER_KEY'] = 'teXgchdxzL9bYnNDT4HctTfUm'
# credentials['CONSUMER_SECRET'] = 'xkNp4LLXdk4vemHF15Bfd0Qn4H6GCHDqY3jTdwheDKdeG0F1kH'
# credentials['ACCESS_TOKEN'] = '837565120435765248-EUarCt01HCRXkCuIHdrWunkJM96FVoD'
# credentials['ACCESS_SECRET'] = 'w8jLAtFxYCeSSVVCTWfKIv9NNYhOvbOBCRyCY13EGKqcK'

## CREDENTIALS 2 ##
credentials['CONSUMER_KEY'] = 'BcPXZRwvvcWCCMKU0tZsN2EGY'
credentials['CONSUMER_SECRET'] = 'QwGSx6nFRYigJsszAGpZemDcgls1kp4otA3mEmsczwVi7u5y5J'
credentials['ACCESS_TOKEN'] = '1050660458254876672-xos6g6ejyVBVvOJVg88VarkIQ8PqF9'
credentials['ACCESS_SECRET'] = '2EPXVunPLsG6CQBBMBXh6AbTpmdC9LWr5MjDIaga2N0xz'


In [6]:
!pip install twython
# Import the Twython class
from twython import Twython  

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# Create our query
query = {'q': 'learn python',  
        'result_type': 'popular',
        'count': 10,
        'lang': 'en',
        }


In [7]:
!pip install twitter
from twitter import * 

# create Twitter API object
twitter = Twitter(auth = OAuth(creds['ACCESS_TOKEN'],
                  creds['ACCESS_SECRET'],
                  creds['CONSUMER_KEY'],
                  creds['CONSUMER_SECRET']))

In [8]:
!pip install tweepy
import tweepy
auth = tweepy.OAuthHandler(creds['CONSUMER_KEY'],
                  creds['CONSUMER_SECRET'])
auth.set_access_token(creds['ACCESS_TOKEN'],
                  creds['ACCESS_SECRET'])
api = tweepy.API(auth)

In [9]:
import pandas as pd

# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}  
for status in python_tweets.search(**query)['statuses']:  
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])

# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)  
df.sort_values(by='favorite_count', inplace=True, ascending=False)  
df.head(5)

,user,date,text,favorite_count
2,CiscoDevNet,Fri May 10 21:34:59 +0000 2019,Ansible is an amazing open source tool for get...,630
1,msdevUK,Fri May 17 16:38:01 +0000 2019,An introduction to machine learning with Pytho...,51
0,adafruit,Sun May 19 13:00:47 +0000 2019,#3Dprinted case for PyPortal! Build a portable...,12


In [10]:
# perform user search
results = twitter.users.search(q = '"rob cornilles"')

# loop through each of the users, and print their details
for user in results:
    if user["name"] == 'Rob Cornilles':
        print("@%s (%s): %s" % (user["screen_name"], user["name"], user["location"]))

@RobCornilles (Rob Cornilles): Oregon's 1st CD


In [11]:
follower_count = api.get_user('RobCornilles').followers_count
print(follower_count)

377


## Speaker Names

In [12]:
# Get unique speaker names
train_news_speaker_unique = pd.DataFrame(train_news.speaker.unique(), columns=['speaker'])


# unique speakers in train dataset
train_news_speaker_unique['speaker'] = train_news_speaker_unique['speaker'].str.replace("-", " ")
train_news_speaker_unique.head(2)

,speaker
0,dwayne bohac
1,scott surovell


In [13]:
# Get unique speaker names
test_news_speaker_unique = pd.DataFrame(test_news.speaker.unique(), columns=['speaker'])


# unique speakers in train dataset
test_news_speaker_unique['speaker'] = test_news_speaker_unique['speaker'].str.replace("-", " ")
test_news_speaker_unique.head(2)

,speaker
0,rick perry
1,katrina shankland


In [14]:
# Get unique speaker names
valid_news_speaker_unique = pd.DataFrame(valid_news.speaker.unique(), columns=['speaker'])


# unique speakers in train dataset
valid_news_speaker_unique['speaker'] = valid_news_speaker_unique['speaker'].str.replace("-", " ")
valid_news_speaker_unique.head(2)

,speaker
0,vicky hartzler
1,chain email


In [16]:
# getting all the unique speakers within all the datasets
speakers = pd.concat([train_news_speaker_unique, test_news_speaker_unique, valid_news_speaker_unique]).drop_duplicates().reset_index(drop=True)
speakers.head(10)

,speaker
0,dwayne bohac
1,scott surovell
2,barack obama
3,blog posting
4,charlie crist
5,robin vos
6,republican party texas
7,oregon lottery
8,duey stroebel
9,robert menendez


In [0]:
# # saving data to csv
# speakers.to_csv('speakers_unique.csv')
# !cp speakers_unique.csv drive/My\ Drive/

## Twitter Handles

In [0]:
speakers['twitter_handle'] = ''

In [21]:
# Getting twitter handle for speakers

speakers = speakers[:10]
#train_speaker = train_news

# speaker name
#for i,(index,row) in enumerate(speakers.iterrows()):
#     if i < 2654: continue # skip first n rows
for index, row in speakers.iterrows():
  
    name = speakers.at[index,'speaker']
    #name = name.replace("-", " ").lower()
    print(name)
    #perform user search
    res = twitter.users.search(q = name)
    
    # loop through each of the users, and print their details
    for user in res:
        #print("@%s (%s): %s" % (user["screen_name"], user["name"], user["location"]))
        if user["name"].lower().find(name) == 0:
            print("@%s (%s): %s" % (user["screen_name"], user["name"], user["location"]) + '\n')
            speakers.at[index,'twitter_handle'] = user["screen_name"]
        break
    #print('\n')

speakers.head()

dwayne bohac
@dwayne_bohac (Dwayne Bohac): Houston, TX

scott surovell
@ssurovell (Scott Surovell): Mt. Vernon, Virginia

barack obama
@BarackObama (Barack Obama): Washington, DC

blog posting
charlie crist
@CharlieCrist (Charlie Crist): St. Petersburg, Florida

robin vos
republican party texas
oregon lottery
@oregon_lottery (Oregon Lottery): Oregon

duey stroebel
@SenStroebel (Duey Stroebel): 

robert menendez


,speaker,twitter_handle
0,dwayne bohac,dwayne_bohac
1,scott surovell,ssurovell
2,barack obama,BarackObama
3,blog posting,
4,charlie crist,CharlieCrist


In [22]:
speakers

,speaker,twitter_handle
0,dwayne bohac,dwayne_bohac
1,scott surovell,ssurovell
2,barack obama,BarackObama
3,blog posting,
4,charlie crist,CharlieCrist
5,robin vos,
6,republican party texas,
7,oregon lottery,oregon_lottery
8,duey stroebel,SenStroebel
9,robert menendez,


In [0]:
# saving data to csv
#speakers.to_csv('speakers3.csv')
#!cp speakers3.csv drive/My\ Drive/


## Twitter Followers

In [24]:
twitter = 'https://raw.githubusercontent.com/jems2019/AlternusVera/socialCredibility/input_data/social_cred/speakers_with_twitter.csv'

twitter = pd.read_csv(twitter, error_bad_lines=False).dropna().reset_index(drop=True)

#twitter['twitter_handle'] = '@' + twitter['twitter_handle'].astype(str)
twitter = twitter.drop(['Unnamed: 0'], axis=1)
twitter.head()

# saving data to csv
#speakers.to_csv('twitter_clean.csv')
#!cp twitter_clean.csv drive/My\ Drive/

,speaker,twitter_handle
0,dwayne bohac,dwayne_bohac
1,scott surovell,ssurovell
2,barack obama,BarackObama
3,charlie crist,CharlieCrist
4,oregon lottery,oregon_lottery


In [25]:
twitter.shape

(1880, 2)

In [27]:
twitter = twitter[:10]
# for i,(index,row) in enumerate(twitter.iterrows()):
#     if i < 1157: continue # skip first n rows
for index, row in twitter.iterrows():
    
    name = twitter.at[index,'twitter_handle'].replace(' \u200f',"")
    #name = name.replace("-", " ").lower()
    print(name)
    follower_count = api.get_user(name).followers_count
    print(" ", follower_count)
    twitter.at[index,'followers'] = follower_count


twitter.head(2)

dwayne_bohac
  2119
ssurovell
  5971
BarackObama
  106187374
CharlieCrist
  32226
oregon_lottery
  3543
SenStroebel
  1403
SenatorMenendez
  161379
SenSanders
  8344166
MittRomney
  1975440
DoonesburyComic
  2548


,speaker,twitter_handle,followers
0,dwayne bohac,dwayne_bohac,2119.0
1,scott surovell,ssurovell,5971.0


In [28]:
twitter.head(10)

,speaker,twitter_handle,followers
0,dwayne bohac,dwayne_bohac,2119.0
1,scott surovell,ssurovell,5971.0
2,barack obama,BarackObama,106187374.0
3,charlie crist,CharlieCrist,32226.0
4,oregon lottery,oregon_lottery,3543.0
5,duey stroebel,SenStroebel,1403.0
6,robert menendez,SenatorMenendez ‏,161379.0
7,bernie s,SenSanders,8344166.0
8,mitt romney,MittRomney,1975440.0
9,doonesbury,DoonesburyComic,2548.0


In [0]:
# saving data to csv
#twitter.to_csv('followers2.csv')
#!cp followers2.csv drive/My\ Drive/

## Number of Tweets

In [29]:
twitter = 'https://raw.githubusercontent.com/jems2019/AlternusVera/socialCredibility/input_data/social_cred/twitter_followers.csv'

twitter = pd.read_csv(twitter, error_bad_lines=False).dropna().reset_index(drop=True)

twitter = twitter.drop(['Unnamed: 0'], axis=1)
twitter.head()


,speaker,twitter_handle,followers
0,dwayne bohac,dwayne_bohac,2124
1,scott surovell,ssurovell,5969
2,barack obama,BarackObama,106128237
3,charlie crist,CharlieCrist,32236
4,oregon lottery,oregon_lottery,3539


In [30]:
# number of tweets SenSanders twitter handle has 

user = api.get_user('SenSanders')
user.statuses_count


20151

In [31]:
twitter = twitter[:10]
# for i,(index,row) in enumerate(twitter.iterrows()):
#     if i < 1723: continue # skip first n rows
for index, row in twitter.iterrows():
    
    name = twitter.at[index,'twitter_handle'].replace(' \u200f',"")
    #name = name.replace("-", " ").lower()
    print(name)
    tweets = api.get_user(name).statuses_count
    verified = api.get_user(name).verified
    print(" ", tweets, " ", verified)
    twitter.at[index,'tweets'] = tweets
    
    if (verified):
      twitter.at[index,'verified'] = 1
    else:
      twitter.at[index,'verified'] = 0
    

    


dwayne_bohac
  1232   False
ssurovell
  8970   False
BarackObama
  15625   True
CharlieCrist
  2717   True
oregon_lottery
  4246   False
SenStroebel
  1201   False
SenatorMenendez
  10233   True
SenSanders
  20151   True
MittRomney
  1804   True
DoonesburyComic
  1160   False


In [32]:
twitter.head(5)

,speaker,twitter_handle,followers,tweets,verified
0,dwayne bohac,dwayne_bohac,2124,1232.0,0.0
1,scott surovell,ssurovell,5969,8970.0,0.0
2,barack obama,BarackObama,106128237,15625.0,1.0
3,charlie crist,CharlieCrist,32236,2717.0,1.0
4,oregon lottery,oregon_lottery,3539,4246.0,0.0


In [0]:
# # saving data to csv
# twitter.to_csv('tweets_verified_5.csv')
# !cp tweets_verified_5.csv drive/My\ Drive/

## Match Twitter handles with dataset

In [33]:
twitter = 'https://raw.githubusercontent.com/jems2019/AlternusVera/socialCredibility/input_data/social_cred/followers_tweets_verified.csv'

twitter = pd.read_csv(twitter, error_bad_lines=False).dropna().reset_index(drop=True)

#twitter['twitter_handle'] = '@' + twitter['twitter_handle'].astype(str)
twitter = twitter.drop(['Unnamed: 0'], axis=1)
twitter.head()


,speaker,twitter_handle,followers,tweets,verified
0,dwayne bohac,dwayne_bohac,2124,1230,0
1,scott surovell,ssurovell,5969,8953,0
2,barack obama,BarackObama,106128237,15625,1
3,charlie crist,CharlieCrist,32236,2712,1
4,oregon lottery,oregon_lottery,3539,4240,0


In [35]:
# Source https://stackoverflow.com/questions/46789098/create-new-column-in-dataframe-with-match-values-from-other-dataframe


train_news['twitter_handle'] = train_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['twitter_handle'])
train_news['followers'] = train_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['followers'])
train_news['tweets'] = train_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['tweets'])
train_news['verified'] = train_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['verified'])

train_news.head(2)

# saving data to csv
# train_news.to_csv('train_news_twitter.csv')
# !cp train_news_twitter.csv drive/My\ Drive/

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context,twitter_handle,followers,tweets,verified
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,dwayne_bohac,2124.0,1230.0,0.0
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,ssurovell,5969.0,8953.0,0.0


In [36]:
# Source https://stackoverflow.com/questions/46789098/create-new-column-in-dataframe-with-match-values-from-other-dataframe



test_news['twitter_handle'] = test_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['twitter_handle'])
test_news['followers'] = test_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['followers'])
test_news['tweets'] = test_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['tweets'])
test_news['verified'] = test_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['verified'])

test_news.head(2)

# saving data to csv
# test_news.to_csv('test_news_twitter.csv')
# !cp test_news_twitter.csv drive/My\ Drive/

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context,twitter_handle,followers,tweets,verified
0,11972.json,true,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview,GovernorPerry,354084.0,3577.0,1.0
1,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference,RepShankland,2652.0,2010.0,1.0


In [37]:
# Source https://stackoverflow.com/questions/46789098/create-new-column-in-dataframe-with-match-values-from-other-dataframe


# valid_news['twitter_handle'] = valid_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['twitter_handle'])
# valid_news.head(2)

valid_news['twitter_handle'] = valid_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['twitter_handle'])
valid_news['followers'] = valid_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['followers'])
valid_news['tweets'] = valid_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['tweets'])
valid_news['verified'] = valid_news['speaker'].str.replace("-", " ").map(twitter.set_index('speaker')['verified'])

valid_news.head(2)

# saving data to csv
# valid_news.to_csv('valid_news_twitter.csv')
# !cp valid_news_twitter.csv drive/My\ Drive/

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context,twitter_handle,followers,tweets,verified
0,12134.json,barely-true,We have less Americans working now than in the...,"economy,jobs",vicky-hartzler,U.S. Representative,Missouri,republican,1,0,1,0,0,an interview with ABC17 News,NaN,NaN,NaN,NaN
1,238.json,pants-fire,"When Obama was sworn into office, he DID NOT u...","obama-birth-certificate,religion",chain-email,NaN,NaN,none,11,43,8,5,105,NaN,NaN,NaN,NaN,NaN
